In [1]:
#Download dahulu library
!pip install yellowbrick

                                              0.0/282.6 kB ? eta -:--:--
     ----                                    30.7/282.6 kB 1.4 MB/s eta 0:00:01
     ------------                            92.2/282.6 kB 1.3 MB/s eta 0:00:01
     --------------------                   153.6/282.6 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 282.6/282.6 kB 1.7 MB/s eta 0:00:00
  Using cached matplotlib-3.7.1-cp310-cp310-win_amd64.whl (7.6 MB)
                                              0.0/8.3 MB ? eta -:--:--
                                              0.1/8.3 MB 4.2 MB/s eta 0:00:02
     -                                        0.3/8.3 MB 3.2 MB/s eta 0:00:03
     --                                       0.5/8.3 MB 3.9 MB/s eta 0:00:02
     ---                                      0.6/8.3 MB 3.7 MB/s eta 0:00:03
     ----                                     0.8/8.3 MB 3.8 MB/s eta 0:00:02
     ----                                     0.9/8.3 MB 3.6 MB/s eta 0:00:03
 

In [6]:
#inisiasi library
from yellowbrick.datasets import load_hobbies
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split as tts

In [7]:
corpus = load_hobbies()
X = TfidfVectorizer().fit_transform(corpus.data)
y = LabelEncoder().fit_transform(corpus.target)
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

In [12]:
X_train #X_train dan lainnya sudah berbentuk np.array, dah kutanya mentorku katanya oke

<358x20140 sparse matrix of type '<class 'numpy.float64'>'
	with 86963 stored elements in Compressed Sparse Row format>

In [ ]:
# Build the Model
model_tune = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, input_shape=[window_size], activation="relu"), 
    tf.keras.layers.Dense(10, activation="relu"), 
    tf.keras.layers.Dense(1)
])
# Set the learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
# Initialize the optimizer
optimizer = tf.keras.optimizers.SGD(momentum=0.9)
# Set the training parameters
model_tune.compile(loss="mse", optimizer=optimizer)
# Train the model
history = model_tune.fit(dataset, epochs=100, callbacks=[lr_schedule])
# Define the learning rate array
lrs = 1e-8 * (10 ** (np.arange(100) / 20))

# Set the figure size
plt.figure(figsize=(10, 6))

# Set the grid
plt.grid(True)

# Plot the loss in log scale
plt.semilogx(lrs, history.history["loss"])

# Increase the tickmarks size
plt.tick_params('both', length=10, width=1, which='both')

# Set the plot boundaries
plt.axis([1e-8, 1e-3, 0, 300])